In [1]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/05/26 10:40:14 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlo1)
24/05/26 10:40:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/26 10:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/26 10:40:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'vieclam24h.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

df_exploded = df.select(explode(col("location")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("location").distinct()

# unique_experience_df.show()

# from pyspark.sql.functions import col
unique_values = df.select(col("age")).distinct().collect()

# In ra các giá trị duy nhất
# for row in unique_values:
#     if row.age is None:
#         print("None ne")
#     print(row.age)





#rdd.toDF().select("salary").show(1)

# Salary


In [16]:
# Salary
def parse_salary(salary):
    if salary is None:
        return {"type": 3 }
    elif (salary.strip().lower() == "thoả thuận" or salary.strip().lower() == "thỏa thuận"):
        return {"type": 2 }
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary), "max": float(max_salary) }
        else:
            return {"type": 0, "fixed_value": float(salary)}
# Age
def parse_age(age):
    if age is None:
        return { "type": 0 }
    age = age.strip().lower()
    if "tuổi" in age:
        age = age.replace(" tuổi", "").strip()
        if "-" in age:
            min_age, max_age = age.split("-")
            return { "type": 1, "min": int(min_age), "max": int(max_age) }
        else:
            return {"value": int(age)}
    return { "type": 0 }

# Experience
def transform_experience(df):
    df = df.withColumn("experience", when(col("experience") == "Chưa có kinh nghiệm", 0).otherwise(col("experience")))
    
    df = df.withColumn("experience", regexp_replace(col("experience"), "(Dưới |Hơn |năm)", ""))
    return df

df = transform_experience(df)

# Type
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", 0)
    .when(col("type") == "Bán thời gian cố định", 1)
    .when(col("type") == "Toàn thời gian tạm thời", 2)
    .when(col("type") == "Khác", 3)
    .otherwise(None)
)

# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Description
def clean_description(description):
    return [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in description]
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))

# Role
def classify_role(role):
    if role == "Chuyên viên- nhân viên":
        return 1
    elif role == "Quản lý cấp cao":
        return 2
    elif role == "Cộng tác viên":
        return 3
    elif role == "Chuyên gia":
        return 4
    elif role == "Quản lý nhóm- giám sát":
        return 5
    elif role == "Quản lý cấp trung":
        return 6
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

def normalize_benefit(benefits):
    return list(map(lambda benefit: re.sub(r'^[*\-+]+', '', benefit).strip(), benefits))

benefit_udf = udf(normalize_benefit, ArrayType(StringType()))
df = df.withColumn("benefit", benefit_udf(col("benefit")))

# location
def normalize_location(locations):
    arr = []
    for location in locations:
        # Tách chuỗi bằng dấu ","
        parts = [part.strip() for part in location.split(',') if part.strip()]
        # Loại bỏ chuỗi rỗng và gán giá trị cho province, district và address
        if len(parts) >= 2:
            province = parts[-1]
            district = parts[-2]
            address = ', '.join(parts[:-2])
            arr.append({"province": province, "district": district, "address": address})
        elif len(parts) == 1:
            arr.append ({"province": parts[-1], "district": None, "address": None })
        else: 
            arr.append({"address": None, "district": None, "province": None})
    return arr
    
rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(row["age"]),
    "location": normalize_location(row["location"])
})

def insert_into_mongodb(partition):
    client = pymongo.MongoClient("mongodb://localhost:27017/", username='admin', password='20194856')
    db = client["thesis"]
    collection = db["jobs"]

    for record in partition:
        collection.insert_one(record)

# Insert dữ liệu vào collection "jobs"
rdd.foreachPartition(insert_into_mongodb)

In [37]:
# Salary
def parse_salary(salary):
    if salary is None or salary.lower() == "thỏa thuận":
        return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return json.dumps({"type": 1, "min": float(min_salary), "max": float(max_salary), "fixed_value": None})
        else:
            return json.dumps({"type": 0, "min": None, "max": None, "fixed_value": float(salary)})
    return json.dumps({"type": 2, "min": None, "max": None, "fixed_value": None})

parse_salary_udf = udf(parse_salary, StringType())
df = df.withColumn("salary", parse_salary_udf(col("salary")))
rdd = df.rdd.map(lambda row: row.asDict())

In [34]:
# Experience
def transform_experience(df):
    df = df.withColumn("experience", when(col("experience") == "Chưa có kinh nghiệm", 0).otherwise(col("experience")))
    
    df = df.withColumn("experience", regexp_replace(col("experience"), "(Dưới |Hơn |năm)", ""))
    return df

df = transform_experience(df)

# Hiển thị kết quả
transformed_df.select("experience").show()

+----------+
|experience|
+----------+
|        2 |
|        1 |
|        1 |
|        1 |
|         0|
|         0|
|        2 |
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|         0|
|        1 |
|        1 |
|         0|
|        2 |
|         0|
+----------+
only showing top 20 rows



In [64]:
# Type
# 0. Toàn thời gian cố định
# 1. Bán thời gian cố định
# 2. Toàn thời gian tạm thời
# 3. Khác
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", 0)
    .when(col("type") == "Bán thời gian cố định", 1)
    .when(col("type") == "Toàn thời gian tạm thời", 2)
    .when(col("type") == "Khác", 3)
    .otherwise(None)
)
df.select("type").show(5)


+----+
|type|
+----+
|   0|
|   0|
|   0|
|   0|
|   0|
+----+
only showing top 5 rows



In [3]:
# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))
df.select("expiration").show(5)

+-------------------+
|         expiration|
+-------------------+
|2024-06-15 00:00:00|
|2024-06-15 00:00:00|
|2024-05-31 00:00:00|
|2024-06-15 00:00:00|
|2024-05-21 00:00:00|
+-------------------+
only showing top 5 rows



In [12]:
# Description
def clean_description(description):
    return [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in description]
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))
df.select("description").show(5)


+--------------------+
|         description|
+--------------------+
|[Sửa chữa, bảo dư...|
|[Trực tiếp kiểm t...|
|[Thi công, lắp đặ...|
|[Nhận và xử lý cá...|
|[Tìm kiếm khách h...|
+--------------------+
only showing top 5 rows



In [17]:
# Role
def classify_role(role):
    if role == "Chuyên viên- nhân viên":
        return 1
    elif role == "Quản lý cấp cao":
        return 2
    elif role == "Cộng tác viên":
        return 3
    elif role == "Chuyên gia":
        return 4
    elif role == "Quản lý nhóm- giám sát":
        return 5
    elif role == "Quản lý cấp trung":
        return 6
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

df.select("role").show(50)


+----+
|role|
+----+
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   5|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   2|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   5|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
|   1|
+----+
only showing top 50 rows



In [38]:
# location
def normalize_location(locations):
    arr = []
    for location in locations:
        # Tách chuỗi bằng dấu ","
        parts = [part.strip() for part in location.split(',') if part.strip()]
        # Loại bỏ chuỗi rỗng và gán giá trị cho province, district và address
        if len(parts) >= 2:
            province = parts[-1]
            district = parts[-2]
            address = ', '.join(parts[:-2])
            arr.append({"province": province, "district": district, "address": address})
        elif len(parts) == 1:
            arr.append ({"province": parts[-1], "district": None, "address": None })
        else: 
            arr.append({"address": None, "district": None, "province": None})
    return arr
# Tạo UDF với PySpark
normalize_location_udf = udf(normalize_location)

# Sử dụng UDF để chuẩn hóa trường "location"
df = df.withColumn("location", normalize_location_udf(df["location"]))

# Hiển thị kết quả
# df_exploded.select("location", "address", "district", "province").show(truncate=False)
df.select("location").show(50)


+--------------------+
|            location|
+--------------------+
|[{address=Lầu 6 T...|
|[{address=Cảng Hà...|
|[{address=20 ngõ ...|
|[{address=Số 6 - ...|
|[{address=SỐ 210,...|
|[{address=Tầng 4,...|
|[{address=290 Hai...|
|[{address=, provi...|
|[{address=Lầu 07,...|
|[{address=lô CN3,...|
|[{address=Văn phò...|
|[{address=Cảng Hà...|
|[{address=, provi...|
|[{address=17-19 T...|
|[{address=LK5, Ng...|
|[{address=Thôn Ch...|
|[{address=Số 2 Ng...|
|[{address=Vinhome...|
|[{address=52 – 53...|
|[{address=T18, Ce...|
|[{address=82/99A ...|
|[{address=161 Ung...|
|[{address=15 Đườn...|
|[{address=192 Trầ...|
|[{address=Đường S...|
|[{address=Số 12, ...|
|[{address=Số 12, ...|
|[{address=Chợ Hàn...|
|[{address=79 Tựu ...|
|[{address=, provi...|
|[{address=Cụm Côn...|
|[{address=44 Tạ Q...|
|[{address=Toàn kh...|
|[{address=, provi...|
|[{address=102 Trầ...|
|[{address=GD 1-5,...|
|[{address=, provi...|
|[{address=418 đườ...|
|[{address=Dương N...|
|[{address=144 Cộn...|
|[{address=

In [39]:
df.select("location").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|location                                                                                                                                                                                                                                                                                                                                                                                                                                        